In [ ]:
import numpy as np
import open3d as o3d
import trimesh
from superprimitive_fusion.utils import (
    trimesh_to_o3d,
    get_o3d_colours_from_trimesh,
)
from superprimitive_fusion.mesh_fusion_utils import (
    smooth_normals,
    calc_local_spacing,
    find_cyl_neighbours,
    compute_overlap_set,
    trilateral_shift,
    find_boundary_edges,
    topological_trim,
    merge_nearby_clusters,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'superprimitive_fusion'

### Load meshes and extract data

In [3]:
mesh1 = trimesh.load_mesh('meshes/bottle_1.ply')
mesh2 = trimesh.load_mesh('meshes/bottle_2.ply')

mesh1_o3d = trimesh_to_o3d(mesh1)
mesh2_o3d = trimesh_to_o3d(mesh2)

In [4]:
points1 = np.asarray(mesh1.vertices)
points2 = np.asarray(mesh2.vertices)
pointclouds = (points1, points2)
points = np.vstack(pointclouds)

colours1 = mesh1.visual.vertex_colors
colours2 = mesh2.visual.vertex_colors
colours = np.concat([colours1, colours2])

tree = o3d.geometry.KDTreeFlann(points.T)

mesh1_o3d.compute_vertex_normals()
mesh2_o3d.compute_vertex_normals()

normals1 = np.asarray(mesh1_o3d.vertex_normals)
normals2 = np.asarray(mesh2_o3d.vertex_normals)
normals = np.concat([normals1, normals2], axis=0)

scan_ids = np.concat([np.ones(len(these_points)) * i for (i, these_points) in enumerate(pointclouds)])

#### Illustrate overlaps and mismatches

In [4]:
mesh1_o3d.paint_uniform_color(np.array([0.9, 0, 0]))
mesh2_o3d.paint_uniform_color(np.array([0, 0.9 , 0]))

# o3d.visualization.draw_geometries([mesh1_o3d, mesh2_o3d])

TriangleMesh with 3144 points and 6071 triangles.

In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

redgreen = np.vstack([
    np.tile([0.9, 0, 0], (points1.shape[0], 1)),
    np.tile([0, 0.9, 0], (points2.shape[0], 1))
])
pcd.colors = o3d.utility.Vector3dVector(redgreen)

# o3d.visualization.draw_geometries([pcd])

### Smooth normals and calculate local properties

In [6]:
normals = smooth_normals(points, normals, k=8, T=0.7, n_iters=5)

In [7]:
local_spacing_1, local_density_1 = calc_local_spacing(mesh1.vertices, np.asarray(mesh1_o3d.vertices))
local_spacing_2, local_density_2 = calc_local_spacing(mesh2.vertices, np.asarray(mesh2_o3d.vertices))
local_spacings = (local_spacing_1, local_spacing_2)
local_spacing = np.concat(local_spacings)

local_density = np.concat((local_spacing_1, local_spacing_2))

global_avg_spacing = (1/len(local_spacings)) * np.sum([(1/len(localspacing)) * np.sum(localspacing) for localspacing in local_spacings])

### Calculate overlapping region

In [8]:
h_alpha = 2.5
r_alpha = 2
overlap_idx, overlap_mask = compute_overlap_set(points, normals, local_spacing, scan_ids, h_alpha, r_alpha, tree)

#### Illustrate overlapping region as a pointcloud

In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[~overlap_mask])
pcd.colors = get_o3d_colours_from_trimesh(colours[~overlap_mask])

# o3d.visualization.draw_geometries([pcd])

### Find the edges that should constrain the overlap mesh

In [10]:
tris1 = np.asarray(mesh1_o3d.triangles)
tris2 = np.asarray(mesh2_o3d.triangles)
all_tris = np.concat([tris1, tris2+len(points1)])

nonoverlap_tris = all_tris[~np.all(overlap_mask[all_tris], axis=1)]

In [11]:
boundary_edges = find_boundary_edges(nonoverlap_tris)

#### Illustrate boundary edges

In [12]:
mask = np.zeros(len(points), dtype=bool)
mask[np.unique(boundary_edges)] = True

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[mask])
pcd.colors = get_o3d_colours_from_trimesh(colours[mask])

# o3d.visualization.draw_geometries([pcd])

### Perform trilateral point shifting

In [13]:
trilat_shifted_pts = points
for i in range(5):
    trilat_shifted_pts = trilateral_shift(trilat_shifted_pts, normals, local_spacing, local_density, overlap_idx, tree, r_alpha, h_alpha)

#### Illustrate point shifted result

In [14]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(trilat_shifted_pts)
pcd.colors = get_o3d_colours_from_trimesh(colours)

# o3d.visualization.draw_geometries([pcd])

### Merge nearby clusters of points

In [15]:
cluster_mapping, clustered_overlap_pnts, clustered_overlap_cols, clustered_overlap_nrms = merge_nearby_clusters(
    trilat_shifted_pts=trilat_shifted_pts,
    normals=normals,
    colours=colours,
    overlap_mask=overlap_mask,
    global_avg_spacing=global_avg_spacing,
    h_alpha=h_alpha,
    find_cyl_neighbours=find_cyl_neighbours,
)

In [16]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(clustered_overlap_pnts)
pcd.colors = get_o3d_colours_from_trimesh(200*np.ones_like(clustered_overlap_cols))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points[cluster_mapping==-1])
pcd2.colors = get_o3d_colours_from_trimesh(100*np.zeros((len(points[cluster_mapping==-1]),4)))

# o3d.visualization.draw_geometries([pcd,pcd2])

### Classify vertices and create mapping
Original vertices were stored in `points`, but some have been merged etc. Resulting vertices should be stored in a new array `new_points`, which has a convenient order (overlap, border, and free vertices).

In [17]:
tris1 = np.asarray(mesh1_o3d.triangles)
tris2 = np.asarray(mesh2_o3d.triangles)

tris2_shifted = tris2 + len(points1)
all_tris = np.vstack([tris1, tris2_shifted])

tri_has_overlap_any = overlap_mask[all_tris].any(axis=1)

overlap_any_idx = np.unique(
    all_tris[tri_has_overlap_any]
)

border_mask = np.zeros(len(points), dtype=bool)
border_mask[overlap_any_idx] = True # Add both border verts and overlap verts
border_mask[cluster_mapping!=-1] = False # Remove overlap verts

nonoverlap_nonborder_mask = np.zeros(len(points), dtype=bool)
nonoverlap_nonborder_mask[cluster_mapping==-1] = True
nonoverlap_nonborder_mask[border_mask] = False

n_overlap, n_border, n_free = len(clustered_overlap_pnts), border_mask.sum(), nonoverlap_nonborder_mask.sum()
#                        Overlapping             Border with overlap              Not overlapping or border (free)
new_points  = np.concat([clustered_overlap_pnts, trilat_shifted_pts[border_mask], trilat_shifted_pts[nonoverlap_nonborder_mask]])
new_colours = np.concat([clustered_overlap_cols, colours[border_mask],            colours[nonoverlap_nonborder_mask]])
new_normals = np.concat([clustered_overlap_nrms, normals[border_mask],            normals[nonoverlap_nonborder_mask]])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points)
c = np.concat([230*np.ones((n_overlap,4)), 160*np.ones((n_border,4)), 0*np.ones((n_free,4))])
pcd.colors = get_o3d_colours_from_trimesh(c)

# Plot vertices and their classifications
# o3d.visualization.draw_geometries([pcd])

In [18]:
overlap_idx_from = overlap_idx
overlap_idx_to   = np.array(range(n_overlap))

border_idx_from  = np.array(range(len(points)))[border_mask]
border_idx_to    = np.array(range(n_border)) + n_overlap

free_idx_from    = np.array(range(len(points)))[nonoverlap_nonborder_mask]
free_idx_to      = np.array(range(n_free)) + n_overlap + n_border

mapping = cluster_mapping # contains mappings for overlap area already
mapping[border_idx_from] = border_idx_to
mapping[free_idx_from]   = free_idx_to

### Mesh the overlap zone

In [19]:
pcd = o3d.geometry.PointCloud()
pcd.points  = o3d.utility.Vector3dVector(new_points[:n_overlap])
pcd.colors = get_o3d_colours_from_trimesh(new_colours[:n_overlap])

if new_normals[:n_overlap] is not None:
    pcd.normals = o3d.utility.Vector3dVector(new_normals[:n_overlap])
else:
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
                            radius=2.5 * global_avg_spacing,  # ~10–30 neighbours
                            max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=30)

ball_r   = 1.1 * global_avg_spacing
radii    = o3d.utility.DoubleVector([ball_r, ball_r * 1.5, ball_r * 2.0])

overlap_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
           pcd, radii)

overlap_mesh.remove_duplicated_vertices()
overlap_mesh.remove_duplicated_triangles()
overlap_mesh.remove_degenerate_triangles()
overlap_mesh.remove_non_manifold_edges()
overlap_mesh.compute_vertex_normals()

overlap_mesh.vertex_colors = pcd.colors

### Debug plot: overlap mesh, border verts, free mesh

In [20]:
v = np.asarray(overlap_mesh.vertices)
vidx = np.unique(mapping[boundary_edges])
vidx = vidx[vidx < len(v)]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(v[vidx])
pcd.colors = get_o3d_colours_from_trimesh(np.zeros((len(vidx), 4)))

free_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(new_points),
                                      triangles=o3d.utility.Vector3iVector(mapping[nonoverlap_tris]))
free_mesh.compute_vertex_normals()

# o3d.visualization.draw_geometries([overlap_mesh, pcd, free_mesh])

TriangleMesh with 3571 points and 1068 triangles.

### Trim overlap mesh to the boundary edge loop

In [21]:
mapped_boundary_edges = mapping[boundary_edges]
relevant_boundary_edges = mapped_boundary_edges[np.all(mapped_boundary_edges<len(v), axis=1)]

trimmed_overlap_mesh = topological_trim(overlap_mesh, relevant_boundary_edges)

# o3d.visualization.draw_geometries([trimmed_overlap_mesh,pcd])

In [22]:
trimmed_overlap_tris = np.asarray(trimmed_overlap_mesh.triangles)
fused_mesh_triangles = np.concatenate([trimmed_overlap_tris, mapping[nonoverlap_tris]], axis=0)
fused_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(new_points),
    triangles=o3d.utility.Vector3iVector(fused_mesh_triangles)
)

fused_mesh.remove_unreferenced_vertices()
fused_mesh.remove_duplicated_triangles()
fused_mesh.remove_duplicated_vertices()
fused_mesh.remove_degenerate_triangles()
fused_mesh.remove_non_manifold_edges()

fused_mesh.vertex_colors = get_o3d_colours_from_trimesh(new_colours)

fused_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([fused_mesh])